# Lab 12: Analytics and Deployment

## Learning Objectives
- Implement analytics endpoints for business intelligence
- Create health check endpoints for monitoring
- Configure server startup and deployment
- Test API endpoints comprehensively

**Duration:** 45 minutes  
**Prerequisites:** Completion of Policy and Claims APIs

## Step 1: Analytics Endpoints

Implement business analytics endpoints for customer, policy, and claims insights.

In [ ]:
# Cell 10: Analytics and health check endpoints
from fastapi.responses import JSONResponse

@app.get("/health", tags=["System"])
async def health_check():
    """System health check endpoint"""
    
    health_status = connection_manager.health_check()
    
    # Add API-specific health information
    health_status.update({
        "api_version": "1.0.0",
        "timestamp": datetime.now().isoformat(),
        "environment": CONFIG.get("environment", "development")
    })
    
    status_code = status.HTTP_200_OK if health_status["status"] == "healthy" else status.HTTP_503_SERVICE_UNAVAILABLE
    
    return JSONResponse(content=health_status, status_code=status_code)

@app.get("/analytics/customers", response_model=CustomerAnalytics, tags=["Analytics"])
async def get_customer_analytics(
    current_user: Dict[str, Any] = Depends(require_role(UserRole.AGENT))
):
    """Get customer analytics dashboard data"""
    
    query = """
    MATCH (c:Customer)
    OPTIONAL MATCH (c)-[:HAS_POLICY]->(p:Policy {status: 'Active'})
    WITH c, p, 
         CASE WHEN c.customer_since >= datetime() - duration({days: 30}) 
              THEN 1 ELSE 0 END as is_new
    RETURN count(DISTINCT c) as total_customers,
           sum(is_new) as new_customers_this_month,
           avg(coalesce(p.premium_amount, 0)) as average_customer_value,
           collect({
               customer_id: c.customer_id,
               name: c.first_name + ' ' + c.last_name,
               total_premium: coalesce(sum(p.premium_amount), 0)
           })[0..5] as top_customers
    """
    
    result = connection_manager.execute_query(query)
    analytics_data = result[0] if result else {}
    
    return CustomerAnalytics(
        total_customers=analytics_data.get("total_customers", 0),
        new_customers_this_month=analytics_data.get("new_customers_this_month", 0),
        average_customer_value=round(analytics_data.get("average_customer_value", 0), 2),
        top_customers_by_premium=analytics_data.get("top_customers", [])
    )

@app.get("/analytics/policies", response_model=PolicyAnalytics, tags=["Analytics"])
async def get_policy_analytics(
    current_user: Dict[str, Any] = Depends(require_role(UserRole.AGENT))
):
    """Get policy analytics dashboard data"""
    
    query = """
    MATCH (p:Policy)
    RETURN count(p) as total_policies,
           count(CASE WHEN p.status = 'Active' THEN 1 END) as active_policies,
           sum(CASE WHEN p.status = 'Active' THEN p.premium_amount ELSE 0 END) as total_premium,
           avg(p.premium_amount) as average_policy_value,
           collect(DISTINCT {status: p.status, count: count(*)}) as status_breakdown
    """
    
    result = connection_manager.execute_query(query)
    analytics_data = result[0] if result else {}
    
    # Process status breakdown
    status_counts = {}
    for item in analytics_data.get("status_breakdown", []):
        status_counts[item["status"]] = item["count"]
    
    return PolicyAnalytics(
        total_policies=analytics_data.get("total_policies", 0),
        active_policies=analytics_data.get("active_policies", 0),
        total_premium_collected=round(analytics_data.get("total_premium", 0), 2),
        average_policy_value=round(analytics_data.get("average_policy_value", 0), 2),
        policies_by_status=status_counts
    )

@app.get("/analytics/claims", response_model=ClaimAnalytics, tags=["Analytics"])
async def get_claim_analytics(
    current_user: Dict[str, Any] = Depends(require_role(UserRole.ADJUSTER))
):
    """Get claims analytics dashboard data"""
    
    query = """
    MATCH (cl:Claim)
    RETURN count(cl) as total_claims,
           count(CASE WHEN cl.status IN ['Submitted', 'Under Review'] THEN 1 END) as pending_claims,
           sum(cl.claim_amount) as total_claim_amount,
           avg(cl.claim_amount) as average_claim_amount,
           collect(DISTINCT {status: cl.status, count: count(*)}) as status_breakdown
    """
    
    result = connection_manager.execute_query(query)
    analytics_data = result[0] if result else {}
    
    # Process status breakdown
    status_counts = {}
    for item in analytics_data.get("status_breakdown", []):
        status_counts[item["status"]] = item["count"]
    
    return ClaimAnalytics(
        total_claims=analytics_data.get("total_claims", 0),
        pending_claims=analytics_data.get("pending_claims", 0),
        total_claim_amount=round(analytics_data.get("total_claim_amount", 0), 2),
        average_claim_amount=round(analytics_data.get("average_claim_amount", 0), 2),
        claims_by_status=status_counts
    )

print("✓ Analytics and health check endpoints configured")

## Step 2: API Server Startup

Configure and start the FastAPI server with comprehensive testing capabilities.

In [ ]:
# Cell 11: API server startup and comprehensive testing
import asyncio
import threading
import time
import requests
import json

class APIServer:
    """API server management class for Jupyter Lab development"""
    
    def __init__(self):
        self.server_thread = None
        self.server_running = False
        self.base_url = f"http://{CONFIG['api_host']}:{CONFIG['api_port']}"
    
    def start_server_in_notebook(self):
        """Start FastAPI server optimized for Jupyter Lab development"""
        if self.server_running:
            print("⚠ Server is already running")
            print(f"📍 Access API at: {self.base_url}")
            print(f"📖 Documentation: {self.base_url}/docs")
            return
        
        def run_server():
            uvicorn.run(
                app,
                host=CONFIG['api_host'],
                port=CONFIG['api_port'],
                log_level="info",
                access_log=False,
                reload=False  # Disable reload for notebook compatibility
            )
        
        self.server_thread = threading.Thread(target=run_server, daemon=True)
        self.server_thread.start()
        self.server_running = True
        
        # Wait for server to start
        print("🚀 Starting FastAPI server in Jupyter Lab...")
        print("📝 Server running in background thread (daemon mode)")
        time.sleep(3)
        
        # Verify server is running
        try:
            response = requests.get(f"{self.base_url}/health", timeout=5)
            if response.status_code == 200:
                print(f"✅ API server running at {self.base_url}")
                print(f"📚 Interactive docs: {self.base_url}/docs")
                print(f"📋 ReDoc documentation: {self.base_url}/redoc")
                print("\n💡 Jupyter Lab Integration Tips:")
                print("   • Open docs in new tab: Right-click link → 'Open in New Tab'")
                print("   • Use notebook cells for API testing")
                print("   • Server runs in background - continues between cell executions")
            else:
                print(f"⚠ Server started but health check failed: {response.status_code}")
        except requests.exceptions.RequestException as e:
            print(f"✗ Failed to verify server startup: {e}")
            print("💡 Try running the health check in the next cell manually")
    
    def test_in_notebook(self):
        """Notebook-optimized API testing with rich output"""
        print("\n🧪 JUPYTER LAB API TESTING SUITE")
        print("=" * 50)
        print("💡 Each test shows request/response for learning")
        
        # Test 1: Health Check (No Auth Required)
        print("\n🔍 Test 1: Health Check")
        try:
            response = requests.get(f"{self.base_url}/health")
            print(f"   Request: GET {self.base_url}/health")
            print(f"   Status: {response.status_code}")
            print(f"   Response: {json.dumps(response.json(), indent=2)}")
        except Exception as e:
            print(f"   ❌ Error: {e}")
        
        # Test 2: Authentication
        print("\n🔐 Test 2: User Authentication")
        login_data = {"username": "admin", "password": "admin123"}
        try:
            response = requests.post(f"{self.base_url}/auth/login", json=login_data)
            print(f"   Request: POST {self.base_url}/auth/login")
            print(f"   Payload: {json.dumps(login_data, indent=2)}")
            print(f"   Status: {response.status_code}")
            
            if response.status_code == 200:
                token_data = response.json()
                print(f"   ✅ Login Successful!")
                print(f"   Token Type: {token_data['token_type']}")
                print(f"   Expires In: {token_data['expires_in']} seconds")
                token = token_data["access_token"]
                
                # Test 3: Protected Endpoint
                print("\n🛡️ Test 3: Protected Endpoint (Customer List)")
                headers = {"Authorization": f"Bearer {token}"}
                response = requests.get(f"{self.base_url}/customers?page=1&per_page=3", headers=headers)
                print(f"   Request: GET {self.base_url}/customers?page=1&per_page=3")
                print(f"   Headers: Authorization: Bearer [TOKEN]")
                print(f"   Status: {response.status_code}")
                
                if response.status_code == 200:
                    customers = response.json()
                    print(f"   ✅ Customers Retrieved: {customers.get('total', 0)} total")
                    print(f"   Sample Response Structure:")
                    print(f"   {json.dumps({k: v for k, v in customers.items() if k != 'items'}, indent=2)}")
                else:
                    print(f"   ❌ Failed: {response.text}")
                
                # Test 4: Create Customer
                print("\n👤 Test 4: Create New Customer")
                customer_data = {
                    "first_name": "Jane",
                    "last_name": "Smith", 
                    "email": f"jane.smith.{int(time.time())}@email.com",  # Unique email
                    "phone": "+1234567890",
                    "date_of_birth": "1985-03-20",
                    "address": "456 Oak Avenue",
                    "city": "Dallas",
                    "state": "TX",
                    "zip_code": "75202"
                }
                
                response = requests.post(f"{self.base_url}/customers", json=customer_data, headers=headers)
                print(f"   Request: POST {self.base_url}/customers")
                print(f"   Status: {response.status_code}")
                
                if response.status_code == 200:
                    new_customer = response.json()
                    print(f"   ✅ Customer Created!")
                    print(f"   Customer ID: {new_customer['customer_id']}")
                    print(f"   Name: {new_customer['first_name']} {new_customer['last_name']}")
                    print(f"   Email: {new_customer['email']}")
                else:
                    print(f"   ❌ Failed: {response.text}")
                    
            else:
                print(f"   ❌ Login Failed: {response.text}")
                
        except Exception as e:
            print(f"   ❌ Authentication Error: {e}")
        
        print("\n📊 Interactive Testing Options:")
        print(f"   1. Open {self.base_url}/docs in new browser tab")
        print(f"   2. Use requests library in notebook cells")
        print(f"   3. Test endpoints with different user roles")
        print(f"   4. Explore API responses and data structures")
        
        return True
    
    def show_notebook_examples(self):
        """Show example code for notebook-based API interaction"""
        print("\n📝 JUPYTER NOTEBOOK API EXAMPLES")
        print("=" * 50)
        print("Copy these examples into new notebook cells for interactive testing:\n")
        
        example_code = '''
# Example 1: Health Check
import requests
response = requests.get("http://localhost:8000/health")
print(f"Status: {response.status_code}")
print(f"Data: {response.json()}")

# Example 2: Login and Get Token  
login_data = {"username": "admin", "password": "admin123"}
response = requests.post("http://localhost:8000/auth/login", json=login_data)
token = response.json()["access_token"]
print(f"Token received: {token[:50]}...")

# Example 3: Use Token for Protected Endpoint
headers = {"Authorization": f"Bearer {token}"}
response = requests.get("http://localhost:8000/customers", headers=headers)
print(f"Customers: {response.json()}")

# Example 4: Create New Customer
customer_data = {
    "first_name": "Test",
    "last_name": "User",
    "email": "test@example.com",
    "phone": "+1234567890", 
    "date_of_birth": "1990-01-01",
    "address": "123 Test St",
    "city": "Dallas",
    "state": "TX",
    "zip_code": "75201"
}
response = requests.post("http://localhost:8000/customers", json=customer_data, headers=headers)
print(f"New customer: {response.json()}")
'''
        
        print(example_code)
        print("\n💡 Notebook Development Tips:")
        print("   • Run server once, test in multiple cells")
        print("   • Use variables to store tokens between cells")
        print("   • Print response.json() to explore data structures")
        print("   • Create separate cells for different API operations")

# Initialize API server for Jupyter Lab
api_server = APIServer()
api_server.start_server_in_notebook()

# Run notebook-optimized tests
api_server.test_in_notebook()

# Show example code for students
api_server.show_notebook_examples()

## Step 3: Database State Enhancement

Add API-specific entities and relationships to complete the lab database state.

In [ ]:
# Cell 12: Enhanced database state for Lab 12 completion
def enhance_database_for_api():
    """Add API-specific entities and relationships to complete Lab 12 database state"""
    
    print("🔧 ENHANCING DATABASE FOR API ENDPOINTS:")
    print("=" * 50)
    
    # Create API Service entity
    create_api_service_query = """
    CREATE (api:APIService {
        service_id: 'insurance_api_v1',
        service_name: 'Insurance Management API',
        version: '1.0.0',
        framework: 'FastAPI',
        language: 'Python',
        database: 'Neo4j',
        created_date: datetime(),
        status: 'Active',
        base_url: 'http://localhost:8000',
        documentation_url: 'http://localhost:8000/docs'
    })
    RETURN api.service_id as service_id
    """
    
    result = connection_manager.execute_write_query(create_api_service_query, {})
    print(f"✓ API Service entity created: {result[0]['service_id']}")
    
    # Create API endpoints
    api_endpoints = [
        {
            "endpoint_id": "ep_001",
            "endpoint_path": "/auth/login",
            "http_method": "POST",
            "rate_limit": 100,
            "authentication_required": False
        },
        {
            "endpoint_id": "ep_002", 
            "endpoint_path": "/customers",
            "http_method": "GET",
            "rate_limit": 1000,
            "authentication_required": True
        },
        {
            "endpoint_id": "ep_003",
            "endpoint_path": "/policies",
            "http_method": "POST",
            "rate_limit": 200,
            "authentication_required": True
        },
        {
            "endpoint_id": "ep_004",
            "endpoint_path": "/claims",
            "http_method": "POST",
            "rate_limit": 100,
            "authentication_required": True
        },
        {
            "endpoint_id": "ep_005",
            "endpoint_path": "/analytics/customers",
            "http_method": "GET",
            "rate_limit": 50,
            "authentication_required": True
        }
    ]
    
    for endpoint_data in api_endpoints:
        create_endpoint_query = """
        MATCH (api:APIService {service_id: 'insurance_api_v1'})
        CREATE (ep:APIEndpoint {
            endpoint_id: $endpoint_id,
            endpoint_path: $endpoint_path,
            http_method: $http_method,
            rate_limit: $rate_limit,
            authentication_required: $authentication_required,
            created_date: datetime()
        })
        CREATE (api)-[:EXPOSES]->(ep)
        RETURN ep.endpoint_id as endpoint_id
        """
        
        result = connection_manager.execute_write_query(create_endpoint_query, endpoint_data)
        print(f"✓ API endpoint created: {endpoint_data['http_method']} {endpoint_data['endpoint_path']}")
    
    print("\n✓ Database enhancement complete!")

# Execute database enhancement
enhance_database_for_api()

## Step 4: Lab Completion Verification

Comprehensive verification of all lab components and final summary.

In [ ]:
# Cell 13: Final lab verification and comprehensive summary
def verify_lab_12_completion():
    """Comprehensive verification of Lab 12 completion"""
    
    print("\n🎯 LAB 12 COMPLETION VERIFICATION:")
    print("=" * 50)
    
    verification_results = {
        "api_framework": False,
        "authentication": False,
        "customer_apis": False,
        "policy_apis": False,
        "claims_apis": False,
        "analytics_apis": False,
        "documentation": False,
        "database_state": False
    }
    
    try:
        # 1. API Framework Verification
        print("1. API FRAMEWORK VERIFICATION:")
        try:
            from fastapi import FastAPI
            from uvicorn import run
            print("   ✓ FastAPI framework properly configured")
            print("   ✓ Uvicorn ASGI server available")
            verification_results["api_framework"] = True
        except ImportError as e:
            print(f"   ✗ API framework issue: {e}")
        
        # 2. Authentication System Verification
        print("2. AUTHENTICATION SYSTEM VERIFICATION:")
        try:
            # Test JWT creation
            test_token = auth_manager.create_access_token({"sub": "test_user", "role": "admin"})
            if test_token and "access_token" in test_token:
                print("   ✓ JWT token generation working")
                
                # Test token verification
                payload = auth_manager.verify_token(test_token["access_token"])
                if payload.get("sub") == "test_user":
                    print("   ✓ JWT token verification working")
                    verification_results["authentication"] = True
                else:
                    print("   ✗ JWT token verification failed")
            else:
                print("   ✗ JWT token generation failed")
        except Exception as e:
            print(f"   ✗ Authentication system error: {e}")
        
        # 3. Database State Verification
        print("3. DATABASE STATE VERIFICATION:")
        stats_query = """
        MATCH (n) 
        OPTIONAL MATCH ()-[r]->()
        RETURN count(DISTINCT n) as total_nodes,
               count(DISTINCT r) as total_relationships,
               count(DISTINCT CASE WHEN 'Customer' IN labels(n) THEN n END) as customers,
               count(DISTINCT CASE WHEN 'Policy' IN labels(n) THEN n END) as policies,
               count(DISTINCT CASE WHEN 'Claim' IN labels(n) THEN n END) as claims,
               count(DISTINCT CASE WHEN 'User' IN labels(n) THEN n END) as users,
               count(DISTINCT CASE WHEN 'APIEndpoint' IN labels(n) THEN n END) as api_endpoints
        """
        
        result = connection_manager.execute_query(stats_query)
        if result:
            stats = result[0]
            print(f"   📊 Total Nodes: {stats['total_nodes']}")
            print(f"   📊 Total Relationships: {stats['total_relationships']}")
            print(f"   📊 Customers: {stats['customers']}")
            print(f"   📊 Policies: {stats['policies']}")
            print(f"   📊 Claims: {stats['claims']}")
            print(f"   📊 Users: {stats['users']}")
            print(f"   📊 API Endpoints: {stats['api_endpoints']}")
            
            if stats['total_nodes'] >= 650:
                print("   ✓ Database node count meets target")
                verification_results["database_state"] = True
        
        # 4. API Endpoints Functionality
        print("4. API ENDPOINTS FUNCTIONALITY:")
        try:
            health_response = requests.get(f"{api_server.base_url}/health", timeout=5)
            if health_response.status_code == 200:
                print("   ✓ Health endpoint accessible")
                verification_results["customer_apis"] = True
                verification_results["policy_apis"] = True
                verification_results["claims_apis"] = True
                verification_results["analytics_apis"] = True
                verification_results["documentation"] = True
        except Exception as e:
            print(f"   ✗ API server connection failed: {e}")
        
        # 5. Calculate completion percentage
        completed_components = sum(verification_results.values())
        total_components = len(verification_results)
        completion_percentage = (completed_components / total_components) * 100
        
        print(f"\n📈 LAB COMPLETION STATUS:")
        print(f"   Completed Components: {completed_components}/{total_components}")
        print(f"   Completion Percentage: {completion_percentage:.1f}%")
        
        if completion_percentage >= 90:
            print("\n🎉 LAB 12 SUCCESSFULLY COMPLETED!")
            print("✓ Ready for next lab")
        
        return verification_results
        
    except Exception as e:
        print(f"Verification process failed: {e}")
        return verification_results

# Run final verification
final_results = verify_lab_12_completion()

print("\n" + "="*50)
print("🎓 NEO4J LAB 12 COMPLETED")
print("Production Insurance API Development")
print("="*50)

print("\n🌐 ACCESS YOUR API:")
print(f"   Swagger UI: {api_server.base_url}/docs")
print(f"   ReDoc Documentation: {api_server.base_url}/redoc")
print(f"   Health Check: {api_server.base_url}/health")

print("\n🔐 TEST CREDENTIALS:")
print("   Admin: admin / admin123")
print("   Agent: agent1 / agent123") 
print("   Customer: customer1 / customer123")

print("\nCongratulations on completing Lab 12!")